In [1]:
import pandas as pd

In [15]:
masses_data = pd.read_csv("data/mammographic_masses_data.csv", na_values='?')
masses_data.head()

,BI-RADS,age,shape,margin,density,severity
0,5.0,67.0,3.0,5.0,3.0,1
1,4.0,43.0,1.0,1.0,NaN,1
2,5.0,58.0,4.0,5.0,3.0,1
3,4.0,28.0,1.0,1.0,3.0,0
4,5.0,74.0,1.0,5.0,NaN,1


In [16]:
masses_data.describe()

,BI-RADS,age,shape,margin,density,severity
count,959.000000,956.000000,930.000000,913.000000,885.000000,961.000000
mean,4.300313,55.487448,2.721505,2.796276,2.910734,0.463059
std,0.683469,14.480131,1.242792,1.566546,0.380444,0.498893
min,0.000000,18.000000,1.000000,1.000000,1.000000,0.000000
25%,4.000000,45.000000,2.000000,1.000000,3.000000,0.000000
50%,4.000000,57.000000,3.000000,3.000000,3.000000,0.000000
75%,5.000000,66.000000,4.000000,4.000000,3.000000,1.000000
max,6.000000,96.000000,4.000000,5.000000,4.000000,1.000000


In [17]:
masses_data.isna().sum()

BI-RADS      2
age          5
shape       31
margin      48
density     76
severity     0
dtype: int64

In [18]:
masses_data.dropna(inplace=True)
masses_data.describe()

,BI-RADS,age,shape,margin,density,severity
count,830.000000,830.000000,830.000000,830.000000,830.000000,830.000000
mean,4.338554,55.781928,2.781928,2.813253,2.915663,0.485542
std,0.660689,14.671782,1.242361,1.567175,0.350936,0.500092
min,0.000000,18.000000,1.000000,1.000000,1.000000,0.000000
25%,4.000000,46.000000,2.000000,1.000000,3.000000,0.000000
50%,4.000000,57.000000,3.000000,3.000000,3.000000,0.000000
75%,5.000000,66.000000,4.000000,4.000000,3.000000,1.000000
max,6.000000,96.000000,4.000000,5.000000,4.000000,1.000000


In [19]:
X = masses_data.drop(columns='severity')
y = masses_data['severity']

In [20]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
scaled_features = scaler.fit_transform(X)

In [21]:
scaled_features

array([[ 1.00174977,  0.7650629 ,  0.17563638,  1.39618483,  0.24046607],
       [ 1.00174977,  0.15127063,  0.98104077,  1.39618483,  0.24046607],
       [-0.51273531, -1.89470363, -1.43517241, -1.157718  ,  0.24046607],
       ...,
       [-0.51273531,  0.56046548,  0.98104077,  1.39618483,  0.24046607],
       [ 1.00174977,  0.69686376,  0.98104077,  1.39618483,  0.24046607],
       [-0.51273531,  0.42406719,  0.17563638,  0.11923341,  0.24046607]])

In [28]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

def create_model():
    model = Sequential()
    model.add(Dense(6, input_dim=5, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    return model

In [31]:
from sklearn.model_selection import cross_val_score
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

classifier = KerasClassifier(build_fn=create_model, epochs=100, verbose=0)
cross_val_scores = cross_val_score(classifier, scaled_features, y, cv=10)

/var/folders/lp/tt27ss_946n0yb_8s_snhbhr0000gn/T/ipykernel_61554/3402119956.py:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn=create_model, epochs=100, verbose=0)


0.8313253045082092

In [32]:
cross_val_scores.mean(), cross_val_scores

(0.8313253045082092,
 array([0.79518074, 0.85542166, 0.90361446, 0.84337348, 0.89156628,
        0.79518074, 0.81927711, 0.80722892, 0.86746991, 0.73493975]))

In [35]:
from joblib import dump
dump(classifier, 'model.pkl')

['model.pkl']